In [1]:
import numpy as np
np.random.seed(2023)

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
make_images = False

#n_rows, n_cols, n_channels = 9, 191, 1
#n_rows, n_cols, n_channels = 9, 95, 1
#n_rows, n_cols, n_channels = 9, 47, 1
n_rows, n_cols, n_channels = 9, 63, 1

original_dim = n_rows * n_cols

In [3]:
eta_bins = (0.8, 1.24, 1.56, 1.7, 1.8, 1.98, 2.16, 2.4)
eta_bins = eta_bins[::-1]
pt_bins = (-0.5, -0.365, -0.26, -0.155, -0.07, 0.07, 0.155, 0.26, 0.365, 0.5)
#pt_bins = (-0.5, -0.3333, -0.2333, -0.1333, -0.07, 0.07, 0.1333, 0.2333, 0.3333, 0.5)
nlayers = 16  # 5 (CSC) + 4 (RPC) + 3 (GEM) + 4 (DT)
superstrip_size = 32

discr_pt_cut_high = 14.

### Stuff

In [4]:
# Find 'dblstrip' unit
PATTERN_X_SIZE = 128-1
PATTERN_X_CENTRAL = 64-1
PATTERN_X_DIV = 16
PATTERN_X_SECTOR = 5400//PATTERN_X_DIV

# Find 'quadstrip' unit
PATTERN_X_QUAD_SIZE = 64-1
PATTERN_X_QUAD_CENTRAL = 32-1
PATTERN_X_QUAD_DIV = 32
PATTERN_X_QUAD_SECTOR = 5400//PATTERN_X_QUAD_DIV

nlayers = 16

nzones = 7

valid_layers = np.empty(nzones, dtype=np.object)
valid_layers[0] = [0,2,3,4,7,8,10,11]
valid_layers[1] = [0,2,3,4,7,8,9,10,11]
valid_layers[2] = [0,2,3,4,7,8,9,10]
valid_layers[3] = [0,2,3,4,7,8,9,10]
valid_layers[4] = [0,1,2,3,4,5,7,8,9]
valid_layers[5] = [1,2,3,4,5,6,7,8]
valid_layers[6] = [1,2,3,5,6,7,12,13,14]

layer_names = ['ME1/1', 'ME1/2', 'ME2', 'ME3', 'ME4',
               'RE1', 'RE2', 'RE3', 'RE4',
               'GE1/1', 'GE2/1', 'ME0',
               'MB1', 'MB2', 'MB3', 'MB4']
assert(len(layer_names) == nlayers)

ordered_layer_names = ['MB1',
                       'ME0', 'GE1/1', 'ME1/1',
                       'MB2',
                       'ME1/2', 'RE1',
                       'GE2/1', 'RE2', 'ME2',
                       'MB3',
                       'ME3', 'RE3',
                       'ME4', 'RE4',
                       'MB4',]
assert(len(ordered_layer_names) == nlayers)

from six.moves import range, zip, map, filter

def _find_bin_fast(x, bins):
  x = np.clip(x, bins[1], bins[2]-1e-7)
  binx = (x - bins[1]) / (bins[2] - bins[1]) * bins[0]
  return binx.astype(np.int32)

def find_eta_bin_16(eta, bins=(16,0.8,2.4)):
  return _find_bin_fast(np.abs(eta), bins)

def find_pt_bin_100(pt, bins=(100,-0.5,0.5)):
  return _find_bin_fast(pt, bins)

def find_d0_bin_100(d0, bins=(100,-120,120)):
  return _find_bin_fast(d0, bins)

_get_emtf_layer = lambda x: x[0]

_get_emtf_phi = lambda x: x[1]

def _find_dphi(phi1, phi2):
  dphi = np.float32(0)
  dphi += (phi1 - phi2)
  #dphi /= (np.sinh(1.8587) / np.sinh(np.abs(part_eta)))
  dphi /= PATTERN_X_DIV
  dphi = np.round(dphi)
  return dphi.astype(np.int32)

def _find_dphi_quad(phi1, phi2):
  dphi = np.float32(0)
  dphi += (phi1 - phi2)
  #dphi /= (np.sinh(1.8587) / np.sinh(np.abs(part_eta)))
  dphi /= PATTERN_X_QUAD_DIV
  dphi = np.round(dphi)
  return dphi.astype(np.int32)

### Check patterns

In [5]:
# Load
bankfile = '../test7/pattern_bank_18patt.29.npz'

with np.load(bankfile) as loaded:
  patterns_phi = loaded['patterns_phi']

In [6]:
# Print patterns
for emtf_layer in xrange(nlayers):
  for ieta in xrange(len(eta_bins)-1):
    for ipt in xrange(len(pt_bins)-1):
      if emtf_layer in valid_layers[ieta]:
        ind = (ipt, ieta, emtf_layer)
        print ind, patterns_phi[ind]
        if emtf_layer != 2 and (patterns_phi[ind][2] - patterns_phi[ind][0]) == 0:
          print("[WARNING] The range is zero. Please check!")

(0, 0, 0) [-14  -7  -2]
(1, 0, 0) [-8 -5 -2]
(2, 0, 0) [-5 -3 -1]
(3, 0, 0) [-3 -1  1]
(4, 0, 0) [-2  0  2]
(5, 0, 0) [-1  1  3]
(6, 0, 0) [1 3 5]
(7, 0, 0) [2 5 8]
(8, 0, 0) [ 2  7 14]
(0, 1, 0) [-16 -10  -4]
(1, 1, 0) [-10  -6  -3]
(2, 1, 0) [-6 -4 -2]
(3, 1, 0) [-4 -1  1]
(4, 1, 0) [-2  0  2]
(5, 1, 0) [-1  1  4]
(6, 1, 0) [2 4 6]
(7, 1, 0) [ 3  6 10]
(8, 1, 0) [ 4 10 16]
(0, 2, 0) [-16 -10  -5]
(1, 2, 0) [-11  -7  -4]
(2, 2, 0) [-7 -4 -2]
(3, 2, 0) [-4 -1  1]
(4, 2, 0) [-2  0  2]
(5, 2, 0) [-1  1  4]
(6, 2, 0) [2 4 7]
(7, 2, 0) [ 4  7 11]
(8, 2, 0) [ 5 10 16]
(0, 3, 0) [-18 -12  -7]
(1, 3, 0) [-12  -8  -5]
(2, 3, 0) [-8 -5 -3]
(3, 3, 0) [-4 -2  1]
(4, 3, 0) [-3  0  3]
(5, 3, 0) [-1  2  4]
(6, 3, 0) [3 5 8]
(7, 3, 0) [ 5  8 12]
(8, 3, 0) [ 7 12 18]
(0, 4, 0) [-19 -12  -6]
(1, 4, 0) [-13  -8  -5]
(2, 4, 0) [-8 -5 -3]
(3, 4, 0) [-4 -2  1]
(4, 4, 0) [-3  0  3]
(5, 4, 0) [-1  2  4]
(6, 4, 0) [3 5 8]
(7, 4, 0) [ 5  8 13]
(8, 4, 0) [ 6 12 19]
(0, 4, 1) [-9 -5 -1]
(1, 4, 1) [-6 -4 -2]
(2, 

### Create patterns

In [7]:
# 5 patterns (symmetric), no charge discrimination

definitions = [
  "4,15:15:15:15,7:7:7:7,7:7:7:7,7:7:7:7",
  "3,16:16:14:14,7:7:7:7,8:7:7:6,8:7:7:6",
  "2,18:17:13:12,7:7:7:7,10:7:7:4,10:7:7:4",
  "1,22:19:11:8,7:7:7:7,14:7:7:0,14:7:7:0",
  "0,30:23:7:0,7:7:7:7,14:7:7:0,14:7:7:0",
]

# "15" is assumed to be the center for ME1
get_window_me1 = lambda x: int(x) - 15

# "7" is assumed to be the center for ME2,3,4
get_window_me234 = lambda x: int(x) - 7

all_windows = []

for d in definitions:
  straightness, wd_me1, wd_me2, wd_me3, wd_me4 = d.split(",")
  #print(straightness, wd_me1, wd_me2, wd_me3, wd_me4)

  windows = []
  for wd in [wd_me1]:
    a, b, c, d = wd.split(":")
    a, b, c, d = map(get_window_me1, [a, b, c, d])
    windows.append([a, b, c, d])
  for wd in [wd_me2, wd_me3, wd_me4]:
    a, b, c, d = wd.split(":")
    a, b, c, d = map(get_window_me234, [a, b, c, d])
    windows.append([a, b, c, d])
  
  all_windows.append(windows)
  # end for loop

all_windows = np.asarray(all_windows)
print(all_windows.shape, all_windows)

((5, 4, 4), array([[[  0,   0,   0,   0],
        [  0,   0,   0,   0],
        [  0,   0,   0,   0],
        [  0,   0,   0,   0]],

       [[  1,   1,  -1,  -1],
        [  0,   0,   0,   0],
        [  1,   0,   0,  -1],
        [  1,   0,   0,  -1]],

       [[  3,   2,  -2,  -3],
        [  0,   0,   0,   0],
        [  3,   0,   0,  -3],
        [  3,   0,   0,  -3]],

       [[  7,   4,  -4,  -7],
        [  0,   0,   0,   0],
        [  7,   0,   0,  -7],
        [  7,   0,   0,  -7]],

       [[ 15,   8,  -8, -15],
        [  0,   0,   0,   0],
        [  7,   0,   0,  -7],
        [  7,   0,   0,  -7]]]))


In [8]:
# Use window = (min, med, max)
get_window3 = lambda x: (x[0], (x[0]+x[1])//2, x[1])

all_windows_9patt = []

for i in xrange(9):
  jj = [4, 3, 2, 1, 0, 1, 2, 3, 4]
  j = jj[i]
  
  kk = [2, 2, 2, 2, 0, 0, 0, 0, 0]
  k = kk[i]
  
  l = 0
  
  wd_me1 = all_windows[j,0,k+1], all_windows[j,0,k]  
  wd_me2 = all_windows[j,1,l+3], all_windows[j,1,l]
  wd_me3 = all_windows[j,2,l+3], all_windows[j,2,l]
  wd_me4 = all_windows[j,3,l+3], all_windows[j,3,l]
  wd_me1, wd_me2, wd_me3, wd_me4 = map(get_window3, [wd_me1, wd_me2, wd_me3, wd_me4])
  
  all_windows_9patt.append([wd_me1, wd_me2, wd_me3, wd_me4])
  # end for loop

all_windows_9patt = np.asarray(all_windows_9patt)
print(all_windows_9patt.shape, all_windows_9patt)

((9, 4, 3), array([[[-15, -12,  -8],
        [  0,   0,   0],
        [ -7,   0,   7],
        [ -7,   0,   7]],

       [[ -7,  -6,  -4],
        [  0,   0,   0],
        [ -7,   0,   7],
        [ -7,   0,   7]],

       [[ -3,  -3,  -2],
        [  0,   0,   0],
        [ -3,   0,   3],
        [ -3,   0,   3]],

       [[ -1,  -1,  -1],
        [  0,   0,   0],
        [ -1,   0,   1],
        [ -1,   0,   1]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  1,   1,   1],
        [  0,   0,   0],
        [ -1,   0,   1],
        [ -1,   0,   1]],

       [[  2,   2,   3],
        [  0,   0,   0],
        [ -3,   0,   3],
        [ -3,   0,   3]],

       [[  4,   5,   7],
        [  0,   0,   0],
        [ -7,   0,   7],
        [ -7,   0,   7]],

       [[  8,  11,  15],
        [  0,   0,   0],
        [ -7,   0,   7],
        [ -7,   0,   7]]]))


In [9]:
patterns_phi_run2 = patterns_phi.copy()
patterns_phi_run2 *= 0  # zero out

for emtf_layer in xrange(nlayers):
  for ieta in xrange(len(eta_bins)-1):
    for ipt in xrange(len(pt_bins)-1):
      if emtf_layer in valid_layers[ieta]:
        ind = (ipt, ieta, emtf_layer)
        
        if emtf_layer == 0:   # ME1/1
          wd = all_windows_9patt[ipt, 0]
          patterns_phi_run2[ind] = wd
        elif emtf_layer == 1: # ME1/2
          wd = all_windows_9patt[ipt, 0]
          patterns_phi_run2[ind] = wd
        elif emtf_layer == 2: # ME2
          wd = all_windows_9patt[ipt, 1]
          patterns_phi_run2[ind] = wd
        elif emtf_layer == 3: # ME3
          wd = all_windows_9patt[ipt, 2]
          patterns_phi_run2[ind] = wd
        elif emtf_layer == 4: # ME4
          wd = all_windows_9patt[ipt, 3]
          patterns_phi_run2[ind] = wd
        elif emtf_layer == 5: # RE1
          wd = all_windows_9patt[ipt, 0]
          patterns_phi_run2[ind] = wd
        elif emtf_layer == 6: # RE2
          wd = all_windows_9patt[ipt, 1]
          patterns_phi_run2[ind] = wd
        elif emtf_layer == 7: # RE3
          wd = all_windows_9patt[ipt, 2]
          patterns_phi_run2[ind] = wd
        elif emtf_layer == 8: # RE4
          wd = all_windows_9patt[ipt, 3]
          patterns_phi_run2[ind] = wd
        elif emtf_layer == 9: # GE1/1
          wd = all_windows_9patt[ipt, 0]
          patterns_phi_run2[ind] = wd
        elif emtf_layer == 10: # GE2/1
          wd = all_windows_9patt[ipt, 1]
          patterns_phi_run2[ind] = wd
        elif emtf_layer == 11: # ME0
          wd = all_windows_9patt[ipt, 0]
          patterns_phi_run2[ind] = wd

In [10]:
# Print patterns
for emtf_layer in xrange(nlayers):
  for ieta in xrange(len(eta_bins)-1):
    for ipt in xrange(len(pt_bins)-1):
      if emtf_layer in valid_layers[ieta]:
        ind = (ipt, ieta, emtf_layer)
        print ind, patterns_phi_run2[ind]
        if emtf_layer != 2 and (patterns_phi[ind][2] - patterns_phi[ind][0]) == 0:
          print("[WARNING] The range is zero. Please check!")

(0, 0, 0) [-15 -12  -8]
(1, 0, 0) [-7 -6 -4]
(2, 0, 0) [-3 -3 -2]
(3, 0, 0) [-1 -1 -1]
(4, 0, 0) [0 0 0]
(5, 0, 0) [1 1 1]
(6, 0, 0) [2 2 3]
(7, 0, 0) [4 5 7]
(8, 0, 0) [ 8 11 15]
(0, 1, 0) [-15 -12  -8]
(1, 1, 0) [-7 -6 -4]
(2, 1, 0) [-3 -3 -2]
(3, 1, 0) [-1 -1 -1]
(4, 1, 0) [0 0 0]
(5, 1, 0) [1 1 1]
(6, 1, 0) [2 2 3]
(7, 1, 0) [4 5 7]
(8, 1, 0) [ 8 11 15]
(0, 2, 0) [-15 -12  -8]
(1, 2, 0) [-7 -6 -4]
(2, 2, 0) [-3 -3 -2]
(3, 2, 0) [-1 -1 -1]
(4, 2, 0) [0 0 0]
(5, 2, 0) [1 1 1]
(6, 2, 0) [2 2 3]
(7, 2, 0) [4 5 7]
(8, 2, 0) [ 8 11 15]
(0, 3, 0) [-15 -12  -8]
(1, 3, 0) [-7 -6 -4]
(2, 3, 0) [-3 -3 -2]
(3, 3, 0) [-1 -1 -1]
(4, 3, 0) [0 0 0]
(5, 3, 0) [1 1 1]
(6, 3, 0) [2 2 3]
(7, 3, 0) [4 5 7]
(8, 3, 0) [ 8 11 15]
(0, 4, 0) [-15 -12  -8]
(1, 4, 0) [-7 -6 -4]
(2, 4, 0) [-3 -3 -2]
(3, 4, 0) [-1 -1 -1]
(4, 4, 0) [0 0 0]
(5, 4, 0) [1 1 1]
(6, 4, 0) [2 2 3]
(7, 4, 0) [4 5 7]
(8, 4, 0) [ 8 11 15]
(0, 4, 1) [-15 -12  -8]
(1, 4, 1) [-7 -6 -4]
(2, 4, 1) [-3 -3 -2]
(3, 4, 1) [-1 -1 -1]
(4, 4, 1) [0 

(4, 4, 9) [0 0 0]
(5, 4, 9) [1 1 1]
(6, 4, 9) [2 2 3]
(7, 4, 9) [4 5 7]
(8, 4, 9) [ 8 11 15]
(0, 0, 10) [0 0 0]
(1, 0, 10) [0 0 0]
(2, 0, 10) [0 0 0]
(3, 0, 10) [0 0 0]
(4, 0, 10) [0 0 0]
(5, 0, 10) [0 0 0]
(6, 0, 10) [0 0 0]
(7, 0, 10) [0 0 0]
(8, 0, 10) [0 0 0]
(0, 1, 10) [0 0 0]
(1, 1, 10) [0 0 0]
(2, 1, 10) [0 0 0]
(3, 1, 10) [0 0 0]
(4, 1, 10) [0 0 0]
(5, 1, 10) [0 0 0]
(6, 1, 10) [0 0 0]
(7, 1, 10) [0 0 0]
(8, 1, 10) [0 0 0]
(0, 2, 10) [0 0 0]
(1, 2, 10) [0 0 0]
(2, 2, 10) [0 0 0]
(3, 2, 10) [0 0 0]
(4, 2, 10) [0 0 0]
(5, 2, 10) [0 0 0]
(6, 2, 10) [0 0 0]
(7, 2, 10) [0 0 0]
(8, 2, 10) [0 0 0]
(0, 3, 10) [0 0 0]
(1, 3, 10) [0 0 0]
(2, 3, 10) [0 0 0]
(3, 3, 10) [0 0 0]
(4, 3, 10) [0 0 0]
(5, 3, 10) [0 0 0]
(6, 3, 10) [0 0 0]
(7, 3, 10) [0 0 0]
(8, 3, 10) [0 0 0]
(0, 0, 11) [-15 -12  -8]
(1, 0, 11) [-7 -6 -4]
(2, 0, 11) [-3 -3 -2]
(3, 0, 11) [-1 -1 -1]
(4, 0, 11) [0 0 0]
(5, 0, 11) [1 1 1]
(6, 0, 11) [2 2 3]
(7, 0, 11) [4 5 7]
(8, 0, 11) [ 8 11 15]
(0, 1, 11) [-15 -12  -8]
(1, 1, 11

In [11]:
# Save
outfile = 'pattern_bank_run2.npz'
np.savez_compressed(outfile, patterns_phi=patterns_phi_run2)